# Klassifikation der Anfänge von Kurzgeschichten nach ihrem Informationsgrad

In [1]:
pip install anthropic


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import anthropic

In [3]:
pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

# Loading Data

In [5]:
file_path = 'Kurzgeschichten_Anfaenge_DHD_2025.xlsx'

In [6]:
df = pd.read_excel(file_path)

In [7]:
df.head()

,Autor,Titel,Absatz,Eingangssatz,Person,Gegenstand,Ort,Zeit,Score,Klassifikation
0,Böll,Der Dieb,"Am meisten war ich darüber erstaunt, wie schne...","Am meisten war ich darüber erstaunt, wie schne...",1,1,0,0,2,sehr wenig Information
1,Böll,Aus der »Vorzeit«,Nachdem die Kompanie vom Kasernenhof abgerückt...,Nachdem die Kompanie vom Kasernenhof abgerückt...,1,2,2,0,5,wenig Information
2,Böll,Im Käfig,Irgendeiner stand am Zaun und blickte grübelnd...,Irgendeiner stand am Zaun und blickte grübelnd...,1,2,2,0,5,wenig Information
3,Böll,In guter Hut,"Als ich wach wurde, wollte ich es zuerst gar n...","Als ich wach wurde, wollte ich es zuerst gar n...",1,2,0,0,3,sehr wenig Information
4,Böll,Todesursache: Hakennase,Als der Leutnant Hegemüller in sein Quartier z...,Als der Leutnant Hegemüller in sein Quartier z...,2,2,2,0,6,wenig Information


In [8]:
df.Klassifikation.value_counts()

Klassifikation
sehr wenig Information    25
wenig Information         22
viel Information           8
Name: count, dtype: int64

## Claude SetUp

In [9]:
client = anthropic.Anthropic(
    api_key = os.getenv('ANTHROPIC_API_KEY')
)

# Evaluating

In [10]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

In [11]:
def get_completion_ant(prompt):
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ])
    return message.content[0].text

In [12]:
results = []
for text in df.Eingangssatz:
    snippet = text
    
    prompt = f"""
    #Role
    You are a literary scholar
    
    #Instruction
    Your task is to recognize points of indeterminacy in German-language narrative texts given to you. The opening of a literary narrative has a point of indeterminacy precisely when it is characterized by a lack of information regarding the central character, the subject, the place or the time of the plot.
    Give first an analysis and then select one of the following German labels to classify the degree of the lack of information regarding the central character, the subject, the place or the time of the plot in the given text snippet:
            Label: sehr viel Information
            Label: viel Information
            Label: wenig Information
            Label: sehr wenig Information
    
    #Output
    Use the following output format:
    Analysis: str
    Label: str
    
    #
    Analyze the following German text: {snippet}
    Analysis: 
    Label:
    """
        
    result = get_completion_ant(prompt)
    print(result)
    results.append(result)

Analysis: This opening sentence provides very little concrete information about the narrative's key elements. We don't know who the narrator ("ich") is, what specifically they are referring to, where or when this realization occurred, or what the overall context or subject matter is. The sentence only conveys a sense of surprise at the speed and simplicity of an unspecified event or process. This lack of specificity creates significant points of indeterminacy regarding character, subject, place, and time.

Label: sehr wenig Information
Analysis: This opening sentence provides a moderate amount of information, but still leaves several points of indeterminacy. We learn that there's a character named Renatus, who is likely a soldier or cadet, as he's part of a "Kompanie" (company) and must change into a white drill uniform as per his service regulations. The setting is partly established - we know it's in a military context, with mention of a "Kasernenhof" (barracks courtyard) and a "Stub

In [ ]:
results = []
for text in df.Eingangssatz:
    snippet = text
    
    prompt = f"""
    #Rolle
    Du bist Literaturwissenschaftler
    
    #Instruction
    Deine Aufgabe  ist es, in den Ihnen vorliegenden deutschsprachigen Erzähltexten Unbestimmtheitsstellen zu erkennen. Der Anfang einer literarischen Erzählung ist genau dann unbestimmt, wenn er durch einen Mangel an Informationen über die Hauptfigur, das Thema, den Ort oder die Zeit der Handlung gekennzeichnet ist.
    Führe zunächst eine Analyse durch und wählen dann eines der folgenden deutschen Labels aus, um den Grad des Mangels an Informationen über die Hauptfigur, das Thema, den Ort oder die Zeit der Handlung in dem gegebenen Textausschnitt zu klassifizieren:
            Label: sehr wenig unbestimmt
            Label: wenig unbestimmt
            Label: stark unbestimmt
            Label: sehr stark unbestimmt
    
    #Output
    Verwende das folgende Ausgabeformat::
    Analyse: str
    Label: str
    
    #
    Analysieren den folgenden deutschen Text: : {snippet}
    Analyse: 
    Label:
    """
    
    message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ])
    
    result = message.content[0].text
    print(result)
    results.append(result)

In [ ]:
results_1 = []
for text in results:
    if "sehr wenig unbestimmt" in text:
        results_1.append("sehr viel Information")
    elif "wenig unbestimmt" in text:
        results_1.append("viel Information")
    elif "sehr stark unbestimmt" in text:
        results_1.append("sehr wenig Information")
    elif "stark unbestimmt" in text:
        results_1.append("wenig Information")
    else:
        results_1.append("NaN")

In [13]:
results_1 = []
for text in results:
    if "sehr viel Information" in text:
        results_1.append("sehr viel Information")
    elif "viel Information" in text:
        results_1.append("viel Information")
    elif "sehr wenig Information" in text:
        results_1.append("sehr wenig Information")
    elif "wenig Information" in text:
        results_1.append("wenig Information")
    else:
        results_1.append("NaN")

In [14]:
results_se = pd.Series(results_1)

In [15]:
predictions = results_se
ground_truth = df.Klassifikation

In [16]:
pd.DataFrame(zip(ground_truth, predictions))

,0,1
0,sehr wenig Information,sehr wenig Information
1,wenig Information,wenig Information
2,wenig Information,sehr wenig Information
3,sehr wenig Information,sehr wenig Information
4,wenig Information,wenig Information
5,wenig Information,sehr wenig Information
6,wenig Information,wenig Information
7,wenig Information,wenig Information
8,sehr wenig Information,wenig Information
9,sehr wenig Information,sehr wenig Information


In [17]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.476062,0.490909,0.497928,0.490909


In [18]:
print(classification_report(ground_truth, predictions))

                        precision    recall  f1-score   support

sehr wenig Information       0.58      0.60      0.59        25
      viel Information       0.50      0.12      0.20         8
     wenig Information       0.41      0.50      0.45        22

              accuracy                           0.49        55
             macro avg       0.49      0.41      0.41        55
          weighted avg       0.50      0.49      0.48        55

